## things to consider in Statistics

1- not just use decribe()/ explain the mean or every column
2- make comparison about corelations between columns, heatmap can help.DO REGRESSIONA ANALYSIS
3-create subction: q-test 
4- do a summary

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup


In [1]:
api_uk_naptan = "https://www.transportforireland.ie/transitData/naptan.json"

In [2]:
api_irl_naptan = 'https://naptan.api.dft.gov.uk/v1/access-nodes?dataFormat=csv'

## MEAKE GET PETITIONS :)

In [10]:
#getting json info of N
naptan_irl_json = "https://www.transportforireland.ie/transitData/naptan.json"
response = requests.get(naptan_irl_json)

In [11]:
response

<Response [200]>

In [13]:
response = requests.get(naptan_irl_json)

# Parse the JSON response
data = response.json()

In [22]:
data['features'][0]['properties']

{'AtcoCode': '7010PB003857',
 'CommonName': 'Ballymagrorty',
 'CommonNameGA': 'B Mhic Robhartaigh',
 'ShortCommonName': 'Ballymagrorty',
 'ShortCommonNameGA': 'B Mhic Robhartaigh',
 'NptgLocalityRef': 'E0701003',
 'Easting': 641057,
 'Northing': 920841,
 'Latitude': 55.0334043,
 'Longitude': -7.3577718,
 'StopType': 'BCT',
 'BusStopType': 'CUS',
 'TimingStatus': 'OTH',
 'CompassPoint': 'NW',
 'AdministrativeAreaRef': 701,
 'Status': 'active',
 'CreationDateTime': '1969-12-31T00:00:00.000',
 'ModificationDateTime': '2021-06-01T16:08:18.000'}